<a href="https://colab.research.google.com/github/grosetta/rdkit_study/blob/main/train_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#文件头代码
import sys
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.insert(0,"/content/gdrive/My Drive/Colab Notebooks/python_package/")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
import csv
import itertools
import operator
import numpy as np
import nltk
import os
from rdkit import Chem
from rdkit.Chem import Draw
from IPython import display
from rdkit.Chem import Descriptors
import sys
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Activation,TimeDistributed
from keras.layers import LSTM,GRU
from keras.layers import Embedding
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.layers import Dropout
import random
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras.models import model_from_json
#from make_smile import ziprocess_organic,process_zinc_data
#from make_smile import zinc_data_with_bracket_original,zinc_processed_with_bracket
from keras.layers import Conv1D, MaxPooling1D

# 函数定义

In [35]:
def zinc_data_with_bracket_original():
#将文件的所有smile转存为一个列表，列表中每一个元素是一个smile
    sen_space=[]
    f = open('/content/gdrive/My Drive/data/250k_rndm_zinc_drugs_clean.smi', 'r')

    reader = csv.reader(f)
    for row in reader:
        sen_space.append(row) #此时的sen_space是列表里套列表的形式
    f.close()

    zinc_processed=[]
    for i in range(len(sen_space)):
        word1=sen_space[i]
        zinc_processed.append(word1[0])
    return zinc_processed

In [39]:
def zinc_processed_with_bracket(sen_space):
    all_smile=[]
    length=[]
    end="\n"
    element_table=["C","N","B","O","P","S","F","Cl","Br","I","(",")","=","#"]
    ring=["1","2","3","4","5","6","7","8","9","10"]

    for i in range(len(sen_space)):
        word_space=sen_space[i]
        word=[]
        j=0
        while j<len(word_space):
            word_space1=[]
            if word_space[j]=="[":
                word_space1.append(word_space[j])
                j=j+1
                while word_space[j]!="]":
                    word_space1.append(word_space[j])
                    j=j+1
                word_space1.append(word_space[j])
                word_space2=''.join(word_space1)
                word.append(word_space2)    #将smile的中括号即其内部的字母作为整体添加到word列表中
                j=j+1
            else:
                word_space1.append(word_space[j])

                if j+1<len(word_space):
                    word_space1.append(word_space[j+1])  #如果第j个字符不是最后一个，那就继续遍历
                    word_space2=''.join(word_space1)
                else:
                    word_space1.insert(0,word_space[j-1]) #如果第j个字符是最后一个
                    word_space2=''.join(word_space1)

                if word_space2 not in element_table:   #如果第j个字符不在预定的元素集合里，就照搬到word列表中 
                    word.append(word_space[j])
                    j=j+1
                else:
                    word.append(word_space2)
                    j=j+2


        word.append(end)  #给每个列表元素最后加终止符
        word.insert(0,"&") #给每个列表第一个元素前加&
        len1=len(word)
        length.append(len1) #每个元素（smile各字符隔开）长度添加到一个列表中，和all_smile中一一对应。
        all_smile.append(list(word))
    val=["\n"]
    for i in range(len(all_smile)):
        for j in range(len(all_smile[i])):   #j的取值范围各不相同，受制于各smile长度
            if all_smile[i][j] not in val:
                val.append(all_smile[i][j])

    return val, all_smile

# 代码主体

In [38]:
smile=zinc_data_with_bracket_original()
valcabulary,all_smile=zinc_processed_with_bracket(smile)
print(valcabulary)
print(len(all_smile))

['\n', '&', 'C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '[C@@H]', 'n', '-', '#', 'S', 'Cl', '[O-]', '[C@H]', '[NH+]', '[C@]', 's', 'Br', '/', '[nH]', '[NH3+]', '4', '[NH2+]', '[C@@]', '[N+]', '[nH+]', '\\', '[S@]', '5', '[N-]', '[n+]', '[S@@]', '[S-]', '6', '7', 'I', '[n-]', 'P', '[OH+]', '[NH-]', '[P@@H]', '[P@@]', '[PH2]', '[P@]', '[P+]', '[S+]', '[o+]', '[CH2-]', '[CH-]', '[SH+]', '[O+]', '[s+]', '[PH+]', '[PH]', '8', '[S@@+]']
249456


In [ ]:
smile